In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
interactions_df = pd.read_csv(os.path.join("data", "hotel_data_interactions_df.csv"), index_col=0)
base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display()

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [3]:
from sklearn.preprocessing import StandardScaler

def prepare_users_df(interactions_df):
    categorical_features = ['term', 'rate_plan', 'length_of_stay_bucket','room_segment', 'n_people_bucket', 'weekend_stay']
    interactions_df_encoded = pd.get_dummies(interactions_df, columns=categorical_features)
    interactions_df_encoded = interactions_df_encoded.drop(columns=['item_id'])
    user_features = interactions_df_encoded.groupby('user_id').sum().reset_index()
    user_features.fillna(0, inplace=True)
    user_features.columns = ['user_id'] + ['user_' + col for col in list(user_features.columns)[1:]]
    user_feature_names = list(user_features.columns)[1:]
    user_features[user_feature_names] = user_features[user_feature_names] / user_features[user_feature_names].sum(axis=1).values.reshape(-1, 1)
    return user_features, user_feature_names


users_df, user_features = prepare_users_df(interactions_df)
print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['user_term_WinterVacation', 'user_term_Easter', 'user_term_OffSeason', 'user_term_HighSeason', 'user_term_LowSeason', 'user_term_MayLongWeekend', 'user_term_NewYear', 'user_term_Christmas', 'user_length_of_stay_bucket_[0-1]', 'user_length_of_stay_bucket_[2-3]', 'user_length_of_stay_bucket_[4-7]', 'user_length_of_stay_bucket_[8-inf]', 'user_rate_plan_Standard', 'user_rate_plan_Nonref', 'user_room_segment_[0-160]', 'user_room_segment_[160-260]', 'user_room_segment_[260-360]', 'user_room_segment_[360-500]', 'user_room_segment_[500-900]', 'user_n_people_bucket_[1-1]', 'user_n_people_bucket_[2-2]', 'user_n_people_bucket_[3-4]', 'user_n_people_bucket_[5-inf]', 'user_weekend_stay_True', 'user_weekend_stay_False']


,user_id,user_term_WinterVacation,user_term_Easter,user_term_OffSeason,user_term_HighSeason,user_term_LowSeason,user_term_MayLongWeekend,user_term_NewYear,user_term_Christmas,user_length_of_stay_bucket_[0-1],...,user_room_segment_[160-260],user_room_segment_[260-360],user_room_segment_[360-500],user_room_segment_[500-900],user_n_people_bucket_[1-1],user_n_people_bucket_[2-2],user_n_people_bucket_[3-4],user_n_people_bucket_[5-inf],user_weekend_stay_True,user_weekend_stay_False
0,1,0.015152,0.0,0.113636,0.015152,0.022727,0.000000,0.000000,0.0,0.000000,...,0.143939,0.022727,0.0,0.0,0.000000,0.121212,0.030303,0.015152,0.128788,0.037879
40,50,0.007246,0.0,0.072464,0.050725,0.036232,0.000000,0.000000,0.0,0.000000,...,0.094203,0.072464,0.0,0.0,0.000000,0.028986,0.086957,0.050725,0.130435,0.036232
84,96,0.015152,0.0,0.113636,0.022727,0.007576,0.007576,0.000000,0.0,0.045455,...,0.143939,0.015152,0.0,0.0,0.007576,0.045455,0.098485,0.015152,0.121212,0.045455
102,115,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.083333,0.083333,0.0,0.0,0.083333,0.000000,0.000000,0.083333,0.000000,0.166667
371,706,0.015905,0.0,0.085487,0.031809,0.023857,0.007952,0.001988,0.0,0.049702,...,0.143141,0.017893,0.0,0.0,0.019881,0.025845,0.097416,0.021869,0.097416,0.069583
1383,1736,0.005747,0.0,0.080460,0.034483,0.045977,0.000000,0.000000,0.0,0.040230,...,0.155172,0.011494,0.0,0.0,0.063218,0.068966,0.034483,0.000000,0.074713,0.091954
7301,7779,0.000000,0.0,0.083333,0.000000,0.083333,0.000000,0.000000,0.0,0.000000,...,0.166667,0.000000,0.0,0.0,0.000000,0.000000,0.125000,0.041667,0.125000,0.041667


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [4]:
def prepare_items_df(interactions_df):
    categorical_features = ['term', 'rate_plan', 'length_of_stay_bucket','room_segment', 'n_people_bucket', 'weekend_stay']
    interactions_df_encoded = pd.get_dummies(interactions_df.loc[:, ['item_id'] + categorical_features], columns=categorical_features)
    item_features = interactions_df_encoded.groupby('item_id').sum().reset_index()
    item_features.fillna(0, inplace=True)
    item_features_names = list(item_features.columns)[1:]
    item_features[item_features_names] = item_features[item_features_names] / item_features[item_features_names].sum(axis=1).values.reshape(-1, 1)
    return item_features, item_features_names


items_df, item_features = prepare_items_df(interactions_df)

print(item_features)


display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['term_WinterVacation', 'term_Easter', 'term_OffSeason', 'term_HighSeason', 'term_LowSeason', 'term_MayLongWeekend', 'term_NewYear', 'term_Christmas', 'rate_plan_Standard', 'rate_plan_Nonref', 'length_of_stay_bucket_[0-1]', 'length_of_stay_bucket_[2-3]', 'length_of_stay_bucket_[4-7]', 'length_of_stay_bucket_[8-inf]', 'room_segment_[0-160]', 'room_segment_[160-260]', 'room_segment_[260-360]', 'room_segment_[360-500]', 'room_segment_[500-900]', 'n_people_bucket_[1-1]', 'n_people_bucket_[2-2]', 'n_people_bucket_[3-4]', 'n_people_bucket_[5-inf]', 'weekend_stay_True', 'weekend_stay_False']


,item_id,term_WinterVacation,term_Easter,term_OffSeason,term_HighSeason,term_LowSeason,term_MayLongWeekend,term_NewYear,term_Christmas,rate_plan_Standard,...,room_segment_[160-260],room_segment_[260-360],room_segment_[360-500],room_segment_[500-900],n_people_bucket_[1-1],n_people_bucket_[2-2],n_people_bucket_[3-4],n_people_bucket_[5-inf],weekend_stay_True,weekend_stay_False
0,0,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.000000,0.166667,0.0,0.0,0.0,0.000000,0.000000,0.166667,0.166667,0.000000
1,1,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.166667,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.166667,0.000000
2,2,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.166667,0.000000,0.0,0.0,0.0,0.166667,0.000000,0.000000,0.000000,0.166667
3,3,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.166667,0.000000,0.0,0.0,0.0,0.000000,0.166667,0.000000,0.166667,0.000000
4,4,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.000000,0.000000,0.0,0.0,0.0,0.166667,0.000000,0.000000,0.166667,0.000000
5,5,0.000000,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.000000,0.166667,0.0,0.0,0.0,0.000000,0.000000,0.166667,0.166667,0.000000
6,6,0.000000,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,0.166667,...,0.000000,0.166667,0.0,0.0,0.0,0.000000,0.000000,0.166667,0.166667,0.000000


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)

        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        

        # Generate negative interactions
        users = interactions_df['user_id'].unique()
        items = interactions_df['item_id'].unique()
        
        negative_interactions = []

        for user in users:
            user_items = interactions_df[interactions_df['user_id'] == user]['item_id'].values
            negative_items = np.random.choice(np.setdiff1d(items, user_items), size=int(self.n_neg_per_pos), replace=True)
            for item in negative_items:
                negative_interactions.append((user, item, 0))

        negative_interactions_df = pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])
        negative_interactions_df.drop_duplicates(subset=['user_id', 'item_id'], inplace=True)

        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])

        # Get the input data for the model
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values
    
        self.model.fit(x, y)
        
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        # Score the items
        
        users_df = users_df.merge(self.users_df, on='user_id', how='left')

        # Fill in missing user features with average feature values from self.users_df
        users_df.fillna(self.users_df.mean(), inplace=True)
        items_df, item_features = prepare_items_df(items_df)
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            user_df = pd.DataFrame([user])
            cartesian_df = pd.merge(user_df, items_df, how="cross")

            scores = self.model.predict(cartesian_df.loc[:, self.user_features + item_features].values)

            chosen_ids = np.argsort(-scores)[:n_recommendations]
    
            recommendations = []
            for item_idx in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': cartesian_df.iloc[item_idx]['item_id'],
                        'score': scores[item_idx]
                    }
                )
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df

    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [6]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [7]:
# Fit method
cb_user_item_recommender = ContentBasedUserItemRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [8]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,51.0,0.619751,OffSeason,[2-3],Nonref,[160-260],[3-4],True
1,1.0,22.0,0.614990,OffSeason,[2-3],Standard,[160-260],[3-4],True
2,1.0,55.0,0.579590,OffSeason,[2-3],Nonref,[160-260],[2-2],True
3,1.0,32.0,0.574829,OffSeason,[2-3],Standard,[160-260],[2-2],True
4,1.0,57.0,0.543091,OffSeason,[2-3],Nonref,[160-260],[3-4],False
5,1.0,34.0,0.538330,OffSeason,[2-3],Standard,[160-260],[3-4],False
6,1.0,78.0,0.512695,HighSeason,[2-3],Nonref,[160-260],[3-4],True
7,1.0,62.0,0.508301,OffSeason,[2-3],Nonref,[160-260],[5-inf],True
8,1.0,9.0,0.507812,HighSeason,[2-3],Standard,[160-260],[3-4],True
9,1.0,76.0,0.504150,OffSeason,[4-7],Nonref,[160-260],[3-4],True


# Tuning method

In [9]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [10]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]
    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10
    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None

    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [73]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}


best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)


print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [15:53<00:00, 95.35s/trial, best loss: -0.22598391874735504]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.175492,0.239308,0.049219,0.094502,0.113541,0.133798


Best parameters:
{'n_neg_per_pos': 1.0}


In [11]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  9%|▉         | 9/100 [9:02:21<91:23:53, 3615.75s/trial, best loss: -0.24925941599661447] 
Best parameters:
None


Traceback (most recent call last):
  File "<ipython-input-10-71711443cfb2>", line 31, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "c:\users\adamw\anaconda3\envs\rs-class-env\lib\site-packages\hyperopt\fmin.py", line 540, in fmin
    return trials.fmin(
  File "c:\users\adamw\anaconda3\envs\rs-class-env\lib\site-packages\hyperopt\base.py", line 671, in fmin
    return fmin(
  File "c:\users\adamw\anaconda3\envs\rs-class-env\lib\site-packages\hyperopt\fmin.py", line 586, in fmin
    rval.exhaust()
  File "c:\users\adamw\anaconda3\envs\rs-class-env\lib\site-packages\hyperopt\fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "c:\users\adamw\anaconda3\envs\rs-class-env\lib\site-packages\hyperopt\fmin.py", line 300, in run
    self.serial_evaluate()
  File "c:\users\adamw\anaconda3\envs\rs-class-env\lib\site-packages\hyperopt\fmin.py", line 178, in serial_evaluate
    result = se

In [77]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 100/100 [3:52:09<00:00, 139.29s/trial, best loss: -0.23444773592890394] 


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.051935,0.136796,0.182281,0.227427,0.051935,0.099698,0.118767,0.133414


Best parameters:
{'max_depth': 10.0, 'min_samples_split': 23.0, 'n_estimators': 181.0, 'n_neg_per_pos': 7.0}


In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.001), np.log(10)),
    'max_iter': hp.quniform('max_iter', 100, 2000, 1),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'])
}

best_param_set = tune_recommender(ContentBasedUserItemRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [69]:
cb_user_item_recommender = ContentBasedUserItemRecommender(
    **{'n_neg_per_pos': 1.0})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['ContentBasedUserItemRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,ContentBasedUserItemRecommender,0.049219,0.130007,0.174134,0.239308,0.049219,0.094502,0.113016,0.133788


In [21]:
import time

content_recommender = ContentBasedUserItemRecommender(
    **{'n_neg_per_pos': 1.0})
amazon_recommender = AmazonRecommender()
linear_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 1.0})
svr_recommender = SVRCBUIRecommender(
    **{'n_neg_per_pos': 3.0})
random_recommender = RandomForestCBUIRecommender(
    **{'n_neg_per_pos': 7.0,
       'n_estimators': 181.0,
       'max_depth': 10.0,
       'min_samples_split':  23.0})
xgb_recommender = XGBoostCBUIRecommender(
    **{'n_neg_per_pos': 3.0})

t0 = time.time()

results = []

models = [content_recommender, amazon_recommender, linear_recommender, svr_recommender, random_recommender, xgb_recommender]
for model in models:
    results.append([model.__class__.__name__] + list(evaluate_train_test_split_implicit(
    model, interactions_df, items_df)))

linear_cbui_tts_results = pd.DataFrame(
    results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,ContentBasedUserItemRecommender,0.049219,0.130007,0.175492,0.231500,0.049219,0.094502,0.113541,0.131541
1,AmazonRecommender,0.044128,0.118805,0.160557,0.223354,0.044128,0.086755,0.104216,0.124370
2,LinearRegressionCBUIRecommender,0.049219,0.130007,0.175492,0.240665,0.049219,0.094502,0.113541,0.134402
3,SVRCBUIRecommender,0.059403,0.135098,0.196198,0.260353,0.059403,0.102450,0.128110,0.148615
4,RandomForestCBUIRecommender,0.047522,0.127291,0.177189,0.229464,0.047522,0.091718,0.112746,0.129737
5,XGBoostCBUIRecommender,0.000000,0.000000,0.000000,0.001358,0.000000,0.000000,0.000000,0.000439


In [74]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.044128,0.118805,0.160557,0.223354,0.044128,0.086755,0.104216,0.12437


In [75]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,ContentBasedUserItemRecommender,0.049219,0.130007,0.174134,0.239308,0.049219,0.094502,0.113016,0.133788
1,AmazonRecommender,0.044128,0.118805,0.160557,0.223354,0.044128,0.086755,0.104216,0.124370
